In [1]:
import torch

<img src="../images/img_chapter_03_type_of_tensors.png" style="width:600px;height:200px"/>

Compared to NumPy arrays, PyTorch tensors have a few superpowers, such as the ability to perform very fast operations on graphical processing units (GPUs), distribute operations on multiple devices or machines, and keep track of the graph of computations that created them. These are all  important features when implementing a modern deep learning library.

### Constructing our first tensors

In [13]:
# Creates a one-dimensional tensor of size 3 filled with 1s
a = torch.ones(3)

In [14]:
a

tensor([1., 1., 1.])

In [15]:
a[1]

tensor(1.)

In [16]:
float(a[2])

1.0

In [17]:
a[2] = 2.0
a

tensor([1., 1., 2.])

### Lists to PyTorch tensor

In [22]:
# Using .zeros is just a way to get an appropriately sized array
points = torch.zeros(6)
points

tensor([0., 0., 0., 0., 0., 0.])

In [23]:
# We overwrite those zeros with the values we actually want
points[0] = 4.0
points[1] = 1.0
points[2] = 5.0
points[3] = 3.0
points[4] = 2.0
points[5] = 1.0

In [24]:
points

tensor([4., 1., 5., 3., 2., 1.])

In [25]:
# We can also pass a Python list to the constructor, to the same effect
points = torch.tensor([4.0, 1.0, 5.0, 3.0, 2.0, 1.0])
points

tensor([4., 1., 5., 3., 2., 1.])

In [26]:
float(points[0]), float(points[1])

(4.0, 1.0)

This is OK, although it would be practical to have the first index refer to individual 2D points rather than point coordinates. For this, we can use a 2D tensor:

In [5]:
list_of_lists = [[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]]
points = torch.tensor(list_of_lists)
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [31]:
points.shape

torch.Size([3, 2])

In [37]:
points[0]
points[1, 0]

tensor(5.)

In [33]:
# Initialize a 2D tensor
points = torch.zeros(3, 2)
points

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

### Indexing tensors

In [48]:
some_list = list(range(6))
some_list[:]

[0, 1, 2, 3, 4, 5]

In [49]:
some_list[1:4]
some_list[1:]
some_list[:4]
some_list[:-1]
some_list[1:4:2]

[1, 3]

In [54]:
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [59]:
points[1:, :] # All rows after the first; all columns
points[1:, 0] # All rows after the first; first column
points[None] # Adds a dimension of size 1, just like unsqueeze

tensor([[[4., 1.],
         [5., 3.],
         [2., 1.]]])

### 3D tensors

In [73]:
img_t = torch.randn(3, 5, 5)
weights = torch.tensor([0.2126, 0.7152, 0.0722])
img_t.shape ,weights.shape

(torch.Size([3, 5, 5]), torch.Size([3]))

In [62]:
batch_t = torch.randn(2, 3, 5, 5)

In [68]:
img_gray_naive = img_t.mean(0)
img_gray_naive.shape

torch.Size([5, 5])

In [70]:
batch_gray_naive = batch_t.mean(-3)
batch_gray_naive.shape

torch.Size([2, 5, 5])

But now we have the weight, too. PyTorch will allow us to multiply things that are the same shape, as well as shapes where one operand is of size 1 in a given dimension. It also appends leading dimensions of size 1 automatically. This is a feature called broadcasting. batch_t of shape (2, 3, 5, 5) is multiplied by unsqueezed_weights of shape (3, 1, 1), resulting in a tensor of shape (2, 3, 5, 5), from which we can then sum the third dimension from the end (the three channels):

In [83]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze(-1)
unsqueezed_weights.shape

torch.Size([3, 1, 1])

In [84]:
img_weights = (img_t * unsqueezed_weights)
img_weights.shape

torch.Size([3, 5, 5])

In [85]:
batch_weights = (batch_t * unsqueezed_weights)
batch_weights.shape

torch.Size([2, 3, 5, 5])

### Specifying the numeric type with dtype

The data type specifies the possible values the tensor can hold (integers versus floatingpoint numbers) and the number of bytes per value. The default data type for tensors is 32-bit floating-point.
##### torch.float32 or torch.float: 32-bit floating-point
##### torch.float64 or torch.double: 64-bit, double-precision floating-point
##### torch.float16 or torch.half: 16-bit, half-precision floating-point
##### torch.int8: signed 8-bit integers
##### torch.uint8: unsigned 8-bit integers
##### torch.int16 or torch.short: signed 16-bit integers
##### torch.int32 or torch.int: signed 32-bit integers
##### torch.int64 or torch.long: signed 64-bit integers
##### torch.bool: Boolean

In [99]:
# using dtype as an argument
double_points = torch.ones(10, 2, dtype=torch.double)
short_points = torch.tensor([[1,2], [3,4]], dtype=torch.short)
double_points.dtype, short_points.dtype

(torch.float64, torch.int16)

In [100]:
# using the corresponding casting method
double_points = torch.ones(10, 2).float()
short_points = torch.tensor([[1,2], [3,4]]).long()
double_points.dtype, short_points.dtype

(torch.float32, torch.int64)

In [103]:
# using 'to' method
double_points = torch.ones(10, 2).to(torch.float64)
short_points = torch.tensor([[1,2], [3,4]]).to(torch.uint8)
double_points.dtype, short_points.dtype

(torch.float64, torch.uint8)

As we will see in future chapters, computations happening in neural networks are typically executed with 32-bit floating-point precision. Higher precision, like 64-bit, will not buy improvements in the accuracy of a model and will require more memory and computing time. The 16-bit floating-point, half-precision data type is not present natively in standard CPUs, but it is offered on modern GPUs. It is possible to switch to half-precision to decrease the footprint of a neural network model if needed, with a minor impact on accuracy.
Creating a tensor with integers as arguments, such as using torch.tensor([2, 2]), will create a 64-bit integer tensor by default. As such, we’ll spend most of our time dealing with float32 and int64.

When mixing input types in operations, the inputs are converted to the larger type automatically. Thus, if we want 32-bit computation, we need to make sure all our inputs are (at most) 32-bit:

### The tensor API

In [110]:
a = torch.ones(3, 2)
a_t = torch.transpose(a, 0, 1)
a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

In [112]:
a = torch.ones(3, 2)
a_t = a.transpose(0, 1)
a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

### Managing Tensor's device

In [4]:
points_gpu = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]], device='cuda')
points_gpu

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]], device='cuda:0')

In [7]:
points_gpu_1 = points.to(device='cuda') # using 'to' method

In [18]:
# Multiplication performed on the CPU
points = 2 * points
# Multiplication performed on the GPU
points_gpu = 2 * points.to(device='cuda') 
# also
points_gpu = 2 * points.to(device='cuda', dtype=torch.int32) 

In [11]:
# to decide on which GPU we allocate the tensor 
# by passing a zero-based integer identifying the GPU
points_gpu = points.to(device='cuda:0')

In [12]:
# to move tensor back to CPU
points_cpu = points_gpu.to(device='cpu')

In [13]:
# Or we can use shot methods
points_gpu = points.cuda() # Defaults to GPU index 0
points_gpu = points.cuda(0)
points_gpu = points_gpu.cpu()

In [5]:
# To get a NumPy array out of tensor
points = torch.ones(3, 4)
points_np = points.numpy()
points_np

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)

If the tensor is allocated on the GPU, PyTorch will make a copy of the content of the tensor into a NumPy array allocated on the CPU.

### Serializing Tensors

PyTorch usespickle under the hood to serialize the tensor object, plus dedicated serialization codefor the storage. Here’s how we can save our points tensor to an ourpoints.t file:

In [6]:
torch.save(points, 'points.t')

In [7]:
# As an alternative, we can pass a file descriptor 
# in lieu of the filename:
with open('points_01.t', 'wb') as f:
    torch.save(points, f)

In [8]:
# Loading our points back is similarly a one-liner
points_new = torch.load('points.t')
print(points_new)

# or

with open('points_01.t', 'rb') as f:
    points_new0 = torch.load(f)
    print(points_new0)

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])
tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])


##### Serializing to HDF5 with h5py
HDF5 is a portable, widely supported
format for representing serialized multidimensional arrays, organized in a nested keyvalue
dictionary. Python supports HDF5 through the h5py library (www.h5py.org),
which accepts and returns data in the form of NumPy arrays.

In [11]:
# Serializing to HDF5 with h5py
import h5py
f = h5py.File('../data/p1ch3/ourpoints.hdf5', 'w')
dset = f.create_dataset('coords', data=points.numpy())
f.close()

At this point, we can save our points tensor by converting it to a NumPy array (at no
cost, as we noted earlier) and passing it to the create_dataset function:

Here 'coords' is a key into the HDF5 file. We can have other keys—even nested ones.
One of the interesting things in HDF5 is that we can index the dataset while on disk
and access only the elements we’re interested in.

In [19]:
f = h5py.File('../data/p1ch3/ourpoints.hdf5', 'r')
dset = f['coords']
last_points = dset[-2:]
last_points

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)

In [20]:
last_points = torch.from_numpy(last_points)
last_points
f.close()

In [27]:
# tensor to numpy
last_points = last_points.numpy()
last_points

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)